In [18]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import snscrape.modules.twitter as sntwitter
import nltk

In [21]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [25]:
import string 
import re
import textblob
from textblob import TextBlob

In [28]:
from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

In [29]:
lemmatizer = WordNetLemmatizer()

In [37]:
from wordcloud import ImageColorGenerator
from PIL import Image

In [38]:
import warnings
%matplotlib inline

In [48]:
import datetime
from tqdm.notebook import tqdm_notebook
sns.set_theme(style="whitegrid")

In [87]:
text = input('Enter query text to be matched (or leave it blank by pressing enter): ')
username = input('Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): ')
since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ')
until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ')
count = int(input('Enter max number of tweets or enter -1 to retrieve all possible tweets: '))
retweet = input('Exclude Retweets? (y/n): ')
replies = input('Exclude Replies? (y/n): ')

Enter query text to be matched (or leave it blank by pressing enter)HP printer
Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): 
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter max number of tweets or enter -1 to retrieve all possible tweets: 20000
Exclude Retweets? (y/n): n
Exclude Replies? (y/n): n


In [88]:
def search(text,username,since,until,retweet,replies):
    global filename
    q = text
    if username!='':
        q += f" from:{username}"
    if until=='':
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
        q += f" until:{until}"
    if since=='':
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - datetime.timedelta(days=7), '%Y-%m-%d')
        q += f" since:{since}"
    if retweet == 'y':
        q += f" exclude:retweets"
    if replies == 'y':
        q += f" exclude:replies"
    if username!='' and text!='':
        filename = f"{since}_{until}_{username}_{text}.csv"
    elif username!="":
        filename = f"{since}_{until}_{username}.csv"
    else:
        filename = f"{since}_{until}_{text}.csv"
        print(filename)
    return q

In [89]:
q = search(text,username,since,until,retweet,replies)
# Creating list to append tweet data
tweets_list1 = []
# Using TwitterSearchScraper to scrape data and append tweets to list
if count == -1:
    for i,tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,tweet.hashtags,tweet.replyCount,tweet.retweetCount, tweet.likeCount,tweet.quoteCount,tweet.media])
else:
    with tqdm_notebook(total=count) as pbar:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): #declare a username
            if i>=count: #number of tweets you want to scrape
                break
            tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,tweet.hashtags,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media])
            pbar.update(1)
# Creating a dataframe from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username','Language','Hashtags','ReplyCount','RetweetCount','LikeCount','QuoteCount','Media']) 

2023-03-20_2023-03-27_HP printer.csv


  0%|          | 0/20000 [00:00<?, ?it/s]

In [90]:
tweets_df1.sort_values(by='DateTime',ascending=False)

,DateTime,TweetId,Text,Username,Language,Hashtags,ReplyCount,RetweetCount,LikeCount,QuoteCount,Media
0,2023-03-26 23:38:16+00:00,1640136114286542855,All I want is a printer that actually works.\n...,jaxxdev,en,None,0,0,0,0,None
1,2023-03-26 23:25:12+00:00,1640132826623246341,Hey @HP if you could make scanner / printer ut...,nemesiscafe,en,None,1,0,1,0,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...
2,2023-03-26 22:54:16+00:00,1640125039977590785,INFITONER 414A Toner Cartridges 4 Pack (with C...,unumihai_,en,None,0,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...
3,2023-03-26 22:41:29+00:00,1640121823122665473,Goodbye HP inkjet. Hello #Brother. I will neve...,1johnwatson,en,[Brother],0,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...
4,2023-03-26 22:32:52+00:00,1640119654516674562,Plenty of black toner left but with yellow emp...,kiwibrew,en,None,4,2,6,0,[Photo(previewUrl='https://pbs.twimg.com/media...
...,...,...,...,...,...,...,...,...,...,...,...
764,2023-03-20 02:10:32+00:00,1637637720087437313,"@HPIndia hp 1005 printer,cuttack ,odisha ,yest...",RawAlok,en,None,3,0,0,0,None
765,2023-03-20 01:56:18+00:00,1637634136880869376,256MB Printer Memory for HP Laserjet Pro 400 C...,prosaccofsu,en,None,0,0,0,0,None
766,2023-03-20 01:34:03+00:00,1637628536197627904,Like It👍 from Computers &amp; Accessories in A...,mieniaisacat,en,None,0,0,0,0,None
767,2023-03-20 01:14:04+00:00,1637623507738558466,"https://t.co/5HWhYyjUxb ink cartridge,ink cart...",AaqibGul6,en,None,0,0,0,0,None


In [83]:
tweets_df1.to_csv(f'{filename}',index = False)

In [91]:
import traceback

try:
    tweets_df1.to_csv(f'{filename}', index=False)
    print("File saved successfully!")
except Exception as e:
    print(f"Error saving file: {e}")
    print(traceback.format_exc())


File saved successfully!


In [ ]:
df = 